In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [9]:
num_grid = 5
num_side = 2
scale_factor = 100
c = np.zeros((num_grid+2, num_side))
c[0+1, 1] = 2/scale_factor
c[num_grid - 1 + 1, 0] = 2/scale_factor
for i in range(num_grid -2):
    c[i+2, 0] = 1/scale_factor
    c[i+2, 1] = 1/scale_factor
print(c)

[[ 0.    0.  ]
 [ 0.    0.02]
 [ 0.01  0.01]
 [ 0.01  0.01]
 [ 0.01  0.01]
 [ 0.02  0.  ]
 [ 0.    0.  ]]


In [10]:
def gain_function_(z, alpha):
    return alpha * math.log(1 + math.exp(z/alpha))

def single_population_rate_model_(E_t, u_t, x_t, z_t, J, tau, E_0, t_fac, t_d, t_th, thre_effi, U, dt):
    du = (U - u_t) / t_fac + U * (1 - u_t) * E_t    
    dE = (- E_t + gain_function_((J * u_t * x_t * E_t - 2.3 + E_0 - thre_effi * z_t), alpha)) / tau
    dx = (1 - x_t) / t_d - u_t * x_t * E_t
    dz = (E_t - z_t)/t_th
    
    E_t_ = E_t + dE * dt
    x_t_ = x_t + dx * dt
    u_t_ = u_t + du * dt
    z_t_ = z_t + dz * dt
    return E_t_, u_t_, x_t_, z_t_

E_ini = 0.
u_ini = 1.
x_ini = 0.
z_ini = 0.

#second
tau = 0.13
# depressing t_d > t_fac 
# facilitating t_d < t_fac    ++++   prefrontal cortex (PFC) facilitating excitatory connection
t_d = 2
t_fac = 15

t_th = 0.1
thre_effi = 0.02

alpha = 0.985

U = 0.3
#E_0 = -2.3
J = 8

dt = 0.0001

T = 10 * (1/dt)
intT = int(T)

time = np.zeros(intT)

E_t = np.zeros((intT, num_grid + 2))
u_t = np.zeros((intT, num_grid))
x_t = np.zeros((intT, num_grid))
z_t = np.zeros((intT, num_grid))

out_of_g = np.zeros((intT, num_grid))
input_t = np.zeros((intT, num_grid))

# E_t[time_index, grid_index]
E_t[0, :] = E_ini
# E_t[:, 0] and E_t[:, num_grid + 2] is boundary. this value has no physical meaning
E_t[:, 0] = 0
E_t[:, num_grid] = 0

u_t[0, :] = u_ini
x_t[0, :] = x_ini
z_t[0, :] = z_ini
#print(E_t, u_t, x_t, z_t)

In [14]:
"""
for i in range(num_grid):
    E_0 = 
    out_of_g[0, i] = gain_function_((J * u_ini * x_ini * E_ini + E_0 - thre_effi * z_ini), alpha)
    input_t[0, i] = J * u_t[0] * x_t[0] * E_t[0] + E_0
"""
# this should be improved
E_0 = E_ini
out_of_g[0, :] = gain_function_((J * u_ini * x_ini * E_ini + E_0 - thre_effi * z_ini), alpha)
input_t[0, :] = J * u_ini * x_ini * E_ini + E_0
    
for i in range(intT - 1):
    time[i+1] = i * dt
    for grid_index in range(num_grid):
        E_t[i+1, grid_index+1], u_t[i+1, grid_index], x_t[i+1, grid_index], z_t[i+1, grid_index] \
        = single_population_rate_model_(E_t[i, grid_index+1], u_t[i, grid_index], x_t[i, grid_index], z_t[i, grid_index]\
                                        ,J, tau, c[grid_index, 1] * E_t[i, grid_index] + c[grid_index+2, 0] * E_t[i, grid_index+2]\
                                        , t_fac, t_d, t_th, thre_effi, U, dt)
        print("E_t[{}, {}] {}= model(~ E0 ~) E0= c[{}, 1] {} * E_t[i, grid_index] {}\
        + c[{}, 0] {} * E_t[i, grid_index+2] {}".format(i+1, grid_index+1, E_t[i+1, grid_index+1],grid_index,c[grid_index, 1], E_t[i, grid_index]\
        , grid_index+1, c[grid_index+2, 0], E_t[i, grid_index+2]))
        #out_of_g[i+1] = gain_function_((J * u_t[i+1] * x_t[i+1] * E_t[i+1] + E_0 - thre_effi * z_t[i+1]), alpha)
        #input_t[i+1] = J * u_t[i+1] * x_t[i+1] * E_t[i+1] + E_0
    key = input("progress")

E_t[1, 1] 7.001392606399197e-05= model(~ E0 ~) E0= c[0, 1] 0.0 * E_t[i, grid_index] 0.0        + c[1, 0] 0.01 * E_t[i, grid_index+2] 0.0
E_t[1, 2] 7.001392606399197e-05= model(~ E0 ~) E0= c[1, 1] 0.02 * E_t[i, grid_index] 0.0        + c[2, 0] 0.01 * E_t[i, grid_index+2] 0.0
E_t[1, 3] 7.001392606399197e-05= model(~ E0 ~) E0= c[2, 1] 0.01 * E_t[i, grid_index] 0.0        + c[3, 0] 0.01 * E_t[i, grid_index+2] 0.0
E_t[1, 4] 7.001392606399197e-05= model(~ E0 ~) E0= c[3, 1] 0.01 * E_t[i, grid_index] 0.0        + c[4, 0] 0.02 * E_t[i, grid_index+2] 0.0
E_t[1, 5] 7.001392606399197e-05= model(~ E0 ~) E0= c[4, 1] 0.01 * E_t[i, grid_index] 0.0        + c[5, 0] 0.0 * E_t[i, grid_index+2] 0.0
progress
E_t[2, 1] 0.00013997404469913611= model(~ E0 ~) E0= c[0, 1] 0.0 * E_t[i, grid_index] 0.0        + c[1, 0] 0.01 * E_t[i, grid_index+2] 7.001392606399197e-05
E_t[2, 2] 0.0001399741397710839= model(~ E0 ~) E0= c[1, 1] 0.02 * E_t[i, grid_index] 7.001392606399197e-05        + c[2, 0] 0.01 * E_t[i, grid_inde

KeyboardInterrupt: 

In [37]:
plt.clf()
fig = plt.figure(figsize = (40, 10), dpi = 600)
for index_grid in range(num_grid):
    plt.plot(time, E_t[:,index_grid+1], label="E_{}".format(index_grid+1))
plt.plot(time, u_t[:,1], label="u")
plt.plot(time, x_t[:,1], label="x")
#plt.plot(time, z_t[:,1], label="threshold")
plt.legend(loc="lower right")
plt.show()